In [1]:
import scanpy as sc

# Prepare training data

In [2]:
data = sc.read_h5ad("10_1016_j_cell_2021_02_018_adaptive_protein.h5ad")

In [3]:
df = data.to_df()
df["label"] = data.obs["mergedcelltype"].values

In [7]:
df

standardized_protein,CD80,CD86,CD274,CD273,CD275,CD11b,CD137L,CD70,CD40,CD3,...,CD360,CD88,HLA-F,NLRP2,Podocalyxin,CD224,c-Met,CD258,DR3,label
AAACCTGAGACGACGT-1_1,14.0,1.0,9.0,8.0,17.0,30.0,8.0,36.0,23.0,6.0,...,17.0,14.0,6.0,28.0,9.0,4.0,19.0,10.0,17.0,B_Naive
AAACCTGAGCTAGTTC-1_1,13.0,2.0,5.0,11.0,17.0,41.0,20.0,37.0,65.0,4.0,...,14.0,10.0,18.0,42.0,7.0,15.0,24.0,15.0,17.0,B_Naive
AAACCTGCATAGACTC-1_1,20.0,0.0,5.0,8.0,6.0,56.0,20.0,47.0,44.0,4.0,...,19.0,20.0,11.0,24.0,4.0,20.0,14.0,5.0,16.0,B_Mem
AAACCTGCATTAACCG-1_1,5.0,1.0,5.0,4.0,11.0,33.0,12.0,38.0,17.0,1.0,...,9.0,12.0,15.0,30.0,2.0,11.0,15.0,10.0,16.0,B_Naive
AAACCTGGTCATCGGC-1_1,21.0,0.0,6.0,10.0,14.0,67.0,13.0,46.0,9.0,51.0,...,13.0,26.0,18.0,31.0,5.0,14.0,24.0,16.0,35.0,T_Vd2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCACAAGAGGCT-16_4,12.0,1.0,16.0,7.0,27.0,62.0,19.0,58.0,19.0,10.0,...,14.0,36.0,26.0,28.0,7.0,18.0,20.0,17.0,30.0,B_Naive
TTTGTCACACGTCTCT-16_4,5.0,2.0,5.0,7.0,11.0,17.0,11.0,19.0,2.0,59.0,...,5.0,5.0,12.0,13.0,5.0,11.0,10.0,4.0,14.0,CD8_Naive
TTTGTCACAGGTCGTC-16_4,11.0,1.0,5.0,6.0,7.0,7.0,9.0,21.0,3.0,108.0,...,13.0,10.0,6.0,11.0,2.0,7.0,6.0,3.0,10.0,CD8_Naive
TTTGTCAGTAAATGAC-16_4,5.0,2.0,8.0,7.0,4.0,11.0,8.0,19.0,1.0,33.0,...,5.0,16.0,10.0,19.0,7.0,48.0,11.0,4.0,12.0,CD4_Mem


# Setup Genetic Algorithm

Source: https://github.com/renatoosousa/GeneticAlgorithmForFeatureSelection

In [4]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from deap import creator, base, tools, algorithms
import sys

# Parameter settings
n_pop = 6
n_gen = 20

In [5]:
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def getFitness(individual, X, y):
    """
    Feature subset fitness function
    """

    if(individual.count(0) != len(individual)):
        # get index with value 0
        cols = [index for index in range(
            len(individual)) if individual[index] == 0]

        # get features subset
        X_parsed = X.drop(X.columns[cols], axis=1)
        X_subset = pd.get_dummies(X_parsed)

        # apply classification algorithm
        clf = LogisticRegression()

        return (avg(cross_val_score(clf, X_subset, y, cv=5)),)
    else:
        return(0,)


def geneticAlgorithm(X, y, n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness, X=X, y=y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    return hof


def bestIndividual(hof, X, y):
    """
    Get the best individual
    """
    maxAccurcy = 0.0
    for individual in hof:
        if(individual.fitness.values[-1] > maxAccurcy):
            maxAccurcy = individual.fitness.values[-1]
            _individual = individual

    _individualHeader = [list(X)[i] for i in range(
        len(_individual)) if _individual[i] == 1]
    return _individual.fitness.values, _individual, _individualHeader

# Main

In [6]:
le = LabelEncoder()
le.fit(df.iloc[:, -1])
y = le.transform(df.iloc[:, -1])
X = df.iloc[:, :-1]

# get accuracy with all features
individual = [1 for i in range(len(X.columns))]
print("Accuracy with all features: \t" +
      str(getFitness(individual, X, y)) + "\n")

Accuracy with all features: 	(0.9398860569483926,)



If we use all features, we get 0.9398860569483926 in accuracy

Now we run the GA model to find the optimal subset of features (nearly reach the all features model performance or even higher) 

In [18]:
# apply genetic algorithm
hof = geneticAlgorithm(X, y, n_pop, n_gen)

gen	nevals	avg     	min    	max     
0  	6     	0.863013	0.78622	0.919203
1  	4     	0.906075	0.895697	0.918183
2  	4     	0.905716	0.895062	0.918183
3  	1     	0.908257	0.90001 	0.918183
4  	4     	0.914428	0.899888	0.925005
5  	4     	0.920357	0.915761	0.925005
6  	5     	0.923214	0.920705	0.925005
7  	2     	0.92548 	0.923823	0.930055
8  	6     	0.92897 	0.927184	0.930055
9  	5     	0.925338	0.90433 	0.930536
10 	1     	0.929864	0.928067	0.93069 
11 	4     	0.927523	0.910489	0.931371
12 	2     	0.930178	0.925167	0.931371
13 	6     	0.930364	0.92788 	0.933136
14 	1     	0.932131	0.931043	0.933136
15 	3     	0.931336	0.922333	0.933136
17 	3     	0.933084	0.932824	0.933136
18 	4     	0.932352	0.929042	0.933136
19 	2     	0.932694	0.931095	0.933136
20 	4     	0.932689	0.930451	0.933136


In [19]:
# select the best individual
accuracy, individual, header = bestIndividual(hof, X, y)

`header` is the subset features that gives the best performance

Now we test again with that subset features

In [20]:
# with feature subset
X = df[header]

In [21]:
clf = LogisticRegression()

scores = cross_val_score(clf, X, y, cv=5)
print("Accuracy with Feature Subset: \t" + str(avg(scores)) + "\n")

Accuracy with Feature Subset: 	0.9331360706505321



Even 0.9331360706505321 < 0.9398860569483926 but we use only 109 proteins out of 193 proteins.

There will be a room for optimization.

In [26]:
header

['CD274',
 'CD273',
 'CD275',
 'CD11b',
 'CD137L',
 'CD40',
 'CD8',
 'CD33',
 'CD11c',
 'CD90',
 'CD45RA',
 'CD123',
 'CD7',
 'CD194',
 'CD4',
 'CD14',
 'CD25',
 'CD45RO',
 'CD279',
 'CD294',
 'CD31',
 'IgM',
 'TCR γ/δ',
 'CD32',
 'CD69',
 'CD62L',
 'CD161',
 'CD152',
 'KLRG1',
 'CD27',
 'CD95',
 'CD1c',
 'CD141',
 'CD57',
 'CD366',
 'CD278',
 'CD39',
 'CX3CR1',
 'CD137',
 'CD303',
 'IgD',
 'CD28',
 'CD38',
 'CD127',
 'CD71',
 'IgG1, κ',
 'IgG2a, κ',
 'IgG2b, κ Isotype Ctrl',
 'Rat_IgG2b_K_Iso',
 'CD252(OX40L)',
 'CD112',
 'CD47',
 'CD154',
 'CD52',
 'TIGIT (VSTM3)',
 'CD326',
 'Podoplanin',
 'CD146',
 'CD324',
 'CD35',
 'CD272',
 'CD96',
 'CD178',
 'CD11a',
 'CD79b',
 'CD235ab',
 'CD370',
 'XCR1',
 'Integrin β7',
 'CD62P',
 'TCR α/β',
 'CD106',
 'FcεRIα',
 'CD41',
 'CD83',
 'CD309',
 'CD124',
 'CD49b',
 'CD45',
 'CD15',
 'CD26',
 'CD193',
 'CD204',
 'CD1a',
 'CD36',
 'CD158',
 'CD207',
 'CD73',
 'TCR Vδ2',
 'CD305',
 'LOX-1',
 'CD158b',
 'CD209',
 'CD158e1',
 'CD337',
 'CD336',
 'CD30